In [1]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [2]:
WINDOW=10
MAXSPEED=110
RESIZE=(64,64)

In [3]:
df=pd.read_csv("./Trails/TW/processed_data_100.csv")

In [4]:
def timezone(ts):
    hour=int(str(ts).split()[1].split(':')[0])
    if 6<=hour<9:
        return 0 #EM
    elif 9<=hour<12:
        return 1 #M
    elif 12<=hour<15:
        return 2 #N
    else:
        return 3 #AN

In [5]:
df['tzones']=df.start_time.apply(timezone)
df=pd.get_dummies(df,columns=['tzones'])
df['roads']=df['high_way']+df['two_way']+df['one_way']
df['amuse']=df['water']+df['park']+df['other_poi']
df['school_exist']=df.school.apply(lambda e: 1 if e>0 else 0)
df['medical_exist']=df.medical.apply(lambda e: 1 if e>0 else 0)
df['speed']*=3.6
df['start_time']=df.start_time.apply(pd.to_datetime)
df['Weakend']=df.DayOfWeek.apply(lambda e: 1 if e in [0,6] else 0)

In [6]:
columns=['start_time', 'patch_path', 'Trail_type', 'trail_no', 
         'mac', 'rsi', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 
         'loudness', 'human_made', 'natural_land',  'roads','amuse', 'school_exist', 'medical_exist','POI_open', 
         'Weakend', 'tzones_0', 'tzones_1', 'tzones_2', 'tzones_3',
         'speed']
dff=df[columns].copy()

In [7]:
feat_cols=['mac', 'rsi', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'loudness', 
           'human_made','natural_land', 'roads', 'amuse', 'school_exist', 'medical_exist','POI_open', 
           'Weakend', 'tzones_0', 'tzones_1', 'tzones_2', 'tzones_3',
           'speed', 'patch_path','Trail_type', 'trail_no']

In [8]:
trails=dff.groupby('trail_no')

In [9]:
seq=[];img=[];spd=[];tno=[];ttype=[]
for trs,data in tqdm(trails):
    data_feat=data.sort_values(by='start_time')[feat_cols].reset_index(drop=True)
    valid_seq=list(data_feat.rolling(window=WINDOW))[9:]

    for prev,nxt in zip(valid_seq[:-1],valid_seq[1:]):
        seq.append(prev.drop(columns=['patch_path','Trail_type', 'trail_no']).values)
        img.append(nxt.patch_path.values[0])
        spd.append(nxt.speed.values[0])
        tno.append(nxt.Trail_type.values[0])
        ttype.append(nxt.trail_no.values[0])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 168/168 [00:06<00:00, 26.69it/s]


In [10]:
dataset={'seq':np.array(seq),
         'img':np.array([np.array(Image.open(img).resize(RESIZE)) for img in img]),
         'spd':np.clip(np.array(spd),0,MAXSPEED),
         'tno':tno,
         'ttype':ttype}

In [11]:
with open('./logs/data/dataset.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#NICE